In [1]:
import dash
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_table as dt
from dash.dependencies import Input, Output, State
import templates
import inspect
import pandas as pd
import numpy as np
import warnings
import dash_bootstrap_components as dbc
import dash_table_experiments as dt
from dash.exceptions import PreventUpdate
warnings.filterwarnings('ignore')

/tmp/ipykernel_2935/1273685219.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/tmp/ipykernel_2935/1273685219.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
/tmp/ipykernel_2935/1273685219.py:5: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table as dt


# Steps
- Get it to display the graphical and table-based ones
- Get it to display right number of questions
- Have answer box display only after generating question
- Fix decimal rounding


In [2]:
classes = []
standards = []
topics = []
grade_levels = []
for name, obj in inspect.getmembers(templates):
    if inspect.isclass(obj):
        classes.append(obj)

In [3]:
for j in range(len(classes)):
    x = classes[j]()
    standard = x.attr1 
    topic = x.attr2 
    topics.append(topic)
    grade_level = x.attr3 
    standards.append(standard)
    grade_levels.append(grade_level)

In [4]:
df = pd.DataFrame({"Class": classes, "Grade_level": grade_levels, "Standard": standards, "Topic": topics})
df.to_csv("4th grade dataframe")

In [5]:
all_topics = list(df['Topic'])
topics = []
for x in all_topics:
    if x not in topics:
        topics.append(x)

In [6]:
markdown_text = '''
Please use this dashboard to select the number and type of SOL questions you would like to practice. Once you select the questions, they will display directly on the webpage.

Please insert your answers directly in the field provided. Good luck!'''

In [7]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

In [8]:
#Option 2
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    [
        html.H1("SOL Math Data Dashboard"),    
        
        dcc.Markdown(children = markdown_text),
        
        html.Div([
            
            html.H3("Grade"),

            dcc.RadioItems(id='grade_level',
                options=[4],
                value=4),

            html.H3("Standard"),

            dcc.RadioItems(id='standard',
                options=['Computation and Estimation','Number and Number Sense', 'Measurement and Geometry', 'Probability, Statistics, Patterns, Functions, and Algebra']),
            
            html.H3("Number of Questions"),

            dcc.Dropdown(id='num_questions',
                options=[{'label': i, 'value': i} for i in range(26)]),
            
            html.Button(id='submit', n_clicks=0, children = 'Create Question(s)', style = {'fontsize': '14'}),

        ], style={'width': '24%', 'float': 'left'}),
         html.Div([
            dcc.Markdown(id='question'),
            dcc.Markdown(id='choices'),
            dcc.Input(id = 'student_answer', type='text', placeholder = "Enter your answer here.", debounce = True),
            html.Div(html.Button('Submit Answer', id= 'submit-val', n_clicks=0)),
            dcc.Markdown(id='feedback'),
            dcc.Store(id = 'correct_answer'),
            dcc.Store(id = 'topic')
        ], style={'width': '74%', 'float': 'right', 'font-weight': 'bold', 'font-size': '20px'})
        
        
    ]
)
#figure out what componenet ID and component property will be here
@app.callback([Output(component_id="question",component_property="children"), 
               Output(component_id="choices",component_property="children"), 
              Output(component_id = 'correct_answer', component_property = 'data'),
              Output(component_id='topic', component_property='data'), 
              Output(component_id = 'submit-val', component_property = "n_clicks")],
                  [Input(component_id='grade_level',component_property="value"),
                   Input(component_id='standard',component_property="value"),
                  Input(component_id='num_questions',component_property="value"),
                  Input(component_id = 'submit', component_property= 'n_clicks')])

def pull_question(grade_level, standard, num_questions, n_clicks, df=df):
    if n_clicks>0:
        temp_df= df.query("Standard == '{standard}' & Grade_level == {grade_level}".format(standard=standard, grade_level=grade_level))
        temp_df = temp_df.sample(n = num_questions, replace= True)
        for i in range(num_questions):
            target_class = temp_df.iloc[i]['Class']
            x= target_class()
            try: 
                question = x.print_question()
                choices= x.print_choices()
                answer = x.correct_letter
                topic = x.attr2
                choices = choices.to_markdown()
                return question, choices, answer, topic, 0
            except:
                question = x.print_question()
                choices = ''
                answer = x.answer
                topic = x.attr2
                return question, choices, answer, topic, 0
    if n_clicks==0:
        return 'Please select a grade level, standard, and number of questions to get started.', "", "", "", 0

@app.callback(Output(component_id="feedback",component_property="children"),
              [Input(component_id = 'correct_answer', component_property = 'data'),
               Input(component_id = 'student_answer', component_property='value'),
              Input(component_id='topic', component_property = 'data'), 
              Input(component_id = 'submit-val', component_property='n_clicks')])

def answer_question(correct_answer, student_answer, topic, n_clicks):
    if n_clicks==0:
        return ""
    if n_clicks == 1:
        if "Fraction Addition/Subtraction" in topic:
            try:
                student_answer= student_answer.split("/")
                num = int(student_answer[0])
                denom = int(student_answer[1])
                student_answer = num/denom
                if student_answer == correct_answer:
                    return "You got the right answer. Nice work!"
                if student_answer != correct_answer:
                    return "You didn't get the right answer this time. Keep trying and don't give up!"
            except:
                return "Next time, please enter your answer as a fraction such as 1/8."
        if "Fraction Ordering" in topic:
            try:
                fraction = student_answer.split(", ")
                numbers= []
                for item in fraction:
                    item = item.split("/")
                    numbers.append(item)
                numbers2= []
                for number in numbers:
                    num = int(number[0])
                    denom= int(number[1])
                    answer = num/denom
                    numbers2.append(answer)
                if numbers2 == correct_answer:
                    return "You got the right answer. Nice work!"
                if numbers2 != correct_answer:
                    return "You didn't get the right answer this time. Keep trying and don't give up!"
            except:
                return "Next time, please enter your answer as a list of fractions separated by commas and a space."
        else:
            if student_answer == correct_answer:
                    return "You got the right answer. Nice work!"
            if student_answer != correct_answer:
                return "You didn't get the right answer this time. Keep trying and don't give up!"
    else:
        return "Please select a new question."
    
if __name__== "__main__":
    app.run_server(mode= 'external', host = "0.0.0.0", debug=True)

Dash app running on http://0.0.0.0:8050/
